In [1]:
from sqlalchemy import Column, Integer, String, Table, ForeignKey
from sqlalchemy.orm import relationship
from sqlmodel import SQLModel, Field, create_engine, Session

# Association tables for many-to-many relationships
candidate_certificates = Table(
    "candidate_certificates",
    SQLModel.metadata,
    Column("candidate_id", ForeignKey("candidates.id"), primary_key=True),
    Column("certificate_id", ForeignKey("certificates.id"), primary_key=True),
)

candidate_qualifications = Table(
    "candidate_qualifications",
    SQLModel.metadata,
    Column("candidate_id", ForeignKey("candidates.id"), primary_key=True),
    Column("qualification_id", ForeignKey("qualifications.id"), primary_key=True),
)

candidate_skills = Table(
    "candidate_skills",
    SQLModel.metadata,
    Column("candidate_id", ForeignKey("candidates.id"), primary_key=True),
    Column("skill_id", ForeignKey("skills.id"), primary_key=True),
)

# Main Candidate table
class Candidate(SQLModel, table=True):
    __tablename__ = "candidates"

    id: int = Field(default=None, primary_key=True)
    name: str = Field(nullable=False, max_length=100)
    email: str = Field(nullable=True, unique=True, max_length=100)
    experience_years: int = Field(nullable=False)

    certificates: list["Certificate"] = relationship(
        "Certificate", secondary=candidate_certificates, back_populates="candidates"
    )
    extra_qualifications: list["Qualification"] = relationship(
        "Qualification", secondary=candidate_qualifications, back_populates="candidates"
    )
    skills: list["Skill"] = relationship(
        "Skill", secondary=candidate_skills, back_populates="candidates"
    )

# Certificate table
class Certificate(SQLModel, table=True):
    __tablename__ = "certificates"

    id: int = Field(default=None, primary_key=True)
    name: str = Field(nullable=False, unique=True, max_length=100)

    candidates: list[Candidate] = relationship(
        "Candidate", secondary=candidate_certificates, back_populates="certificates"
    )

# Qualification table
class Qualification(SQLModel, table=True):
    __tablename__ = "qualifications"

    id: int = Field(default=None, primary_key=True)
    name: str = Field(nullable=False, unique=True, max_length=100)

    candidates: list[Candidate] = relationship(
        "Candidate", secondary=candidate_qualifications, back_populates="extra_qualifications"
    )

# Skill table
class Skill(SQLModel, table=True):
    __tablename__ = "skills"

    id: int = Field(default=None, primary_key=True)
    name: str = Field(nullable=False, unique=True, max_length=100)

    candidates: list[Candidate] = relationship(
        "Candidate", secondary=candidate_skills, back_populates="skills"
    )



ModuleNotFoundError: No module named 'sqlalchemy'

In [ ]:
# Database setup
engine = create_engine("sqlite:///candidates.db")
SQLModel.metadata.create_all(engine)

# Example usage
with Session(engine) as session:
    # Adding a candidate
    candidate = Candidate(name="John Doe", email="john@example.com", experience_years=5)
    certificate = Certificate(name="PMP")
    qualification = Qualification(name="MBA")
    skill = Skill(name="Python")

    # Associate the candidate with certificates, qualifications, and skills
    candidate.certificates.append(certificate)
    candidate.extra_qualifications.append(qualification)
    candidate.skills.append(skill)

    session.add(candidate)
    session.commit()
